In the previous chapter we looked at an example of a three dimensional NIfTI parcellation.  Towards the end of the chapter we visualized the parcellation, but noted that an apparent issue with the affine (or how it was being interpreted) led to an incorrect orientation for the visualization.  Lets explore this issue in a bit more depth.

Lets begin by loading the relevant nifti objects.

In [2]:
import nibabel as nib
#this code ensures that we can navigate the WiMSE repo across multiple systems
import subprocess
import os
#get top directory path of the current git repository, under the presumption that 
#the notebook was launched from within the repo directory
gitRepoPath=subprocess.check_output(['git', 'rev-parse', '--show-toplevel']).decode('ascii').strip()

#move to the top of the directory
os.chdir(gitRepoPath)

#establish path to new nifti file
newAtlasPath=os.path.join(gitRepoPath,'exampleData','renumberedAtlas.nii.gz')   
#load it as an object
atlasImg = nib.load(newAtlasPath)

#establish path to t1
t1Path='/Users/plab/Documents/JupyerData/proj-5c50b6f12daf2e0032f880eb/sub-100206/dt-neuro-anat-t1w.tag-acpc_aligned.tag-brain_extracted.id-5c57072befbc2800526291bb/t1.nii.gz'
#import the data
t1img = nib.load(t1Path)

Lets take a quick look at our to remind us of its orientation.

In [4]:
from niwidgets import NiftiWidget

atlas_widget = NiftiWidget(t1img)

atlas_widget.nifti_plotter(colormap='nipy_spectral')

<Figure size 432x288 with 0 Axes>

interactive(children=(IntSlider(value=72, continuous_update=False, description='x', max=144), IntSlider(value=…

This appears to be fine, though its worth noting we wouldn't be able to discern a left/right flip (a flip along the X axis) with this method.

Lets see if we can apply some rotations to the atlas image to get it oriented correctly.

In [ ]:
def 

In [3]:
#get affine from atlas
atlasImgAff=atlasImg.affine

import nilearn
t1Resampled=nilearn.image.resample_img(t1img,target_affine=atlasImgAff,target_shape=atlasImg.shape)

from niwidgets import NiftiWidget
atlas_widget = NiftiWidget(t1Resampled)

atlas_widget.nifti_plotter(colormap='nipy_spectral')

<Figure size 640x480 with 0 Axes>

interactive(children=(IntSlider(value=127, continuous_update=False, description='x', max=255), IntSlider(value…

In [38]:
def fig2Img(figure2convert):
    from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
    from matplotlib.figure import Figure
    import matplotlib
    import numpy as np

    #canvas = FigureCanvas(figure2convert)
    ax = figure2convert.gca()
    ax.axis('off')
    figure2convert.tight_layout(pad=0)

    # To remove the huge white borders
    ax.margins(0)

    figure2convert.canvas.draw()       # draw the canvas, cache the renderer
    

    image_from_plot = np.frombuffer(figure2convert.canvas.tostring_rgb(), dtype=np.uint8)
    image_from_plot = image_from_plot.reshape(figure2convert.canvas.get_width_height()[::-1] + (3,))

    return image_from_plot
    
def multiDimOverlayPlot(xSlice,ySlice,zSlice):
    #import dipy regtools
    from dipy.viz import regtools 
    #import matplotlib functions

    image1=t1Resampled
    image2=atlasImg
    
    import matplotlib.pyplot as plt
    from matplotlib.pyplot import imshow

    sagitalFig=regtools.overlay_slices(image1.get_fdata(),image2.get_fdata(),slice_index=xSlice, slice_type=0)
    coronalFig=regtools.overlay_slices(image1.get_fdata(),image2.get_fdata(),slice_index=ySlice, slice_type=1)
    axialFig=regtools.overlay_slices(image1.get_fdata(),image2.get_fdata(),slice_index=zSlice, slice_type=2)
   
    
#multiDimOverlayPlot(t1Resampled,atlasImg,132,132,132)
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import IntSlider

interact(multiDimOverlayPlot, \
         xSlice=IntSlider(min=0, max=256, step=1,continuous_update=False),  \
         ySlice=IntSlider(min=0, max=256, step=1,continuous_update=False), \
         zSlice=IntSlider(min=0, max=256, step=1,continuous_update=False)
         )

interactive(children=(IntSlider(value=0, continuous_update=False, description='xSlice', max=256), IntSlider(va…

<function __main__.multiDimOverlayPlot(xSlice, ySlice, zSlice)>